<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

## Introduction

This is an analysis done on Yelp data, in particular the Phoenix Area.

The steps used to analyze the data were:
- Data Cleaning
- Exploratory Data Analysis
- Data modeling and export
- NLP Analysis

## Data Cleaning

### Initial Wrangling

Import Libraries

In [141]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None
import sys
import geopy
import zipcodes
import numpy as np

Exception ignored in: <function SearchEngine.__del__ at 0x00000223BE5C6E58>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\uszipcode\search.py", line 112, in __del__
    if self.ses:
AttributeError: 'SearchEngine' object has no attribute 'ses'


Columns marked for deletion initially

In [2]:
cols_to_remove = ['Business - Dogs Allowed', 'Business - Address', 'Business - Smoking', 'Business - Restaurant?', 'Business - Outdoor Seating', 'Business - Order at Counter', 'Business - Happy Hour', 'Business - Good For Kids', 'Business - Good For Dancing', 'Business - Corkage', 'Business - Coat Check', 'Business - BYOB/Corkage', 'Business - BYOB', 'Business - Monday Close','Business - Saturday Close','Business - Friday Open','Business - Saturday Open','Business - Monday Open','Business - Sunday Close','Business - Sunday Open','Business - Wednesday Close','Business - Thursday Close','Business - Wednesday Open','Business - Thursday Open','Business - Tuesday Close','Business - Tuesday Open','Business - Friday Close','Number of Records','Business - Accepts Insurance','Business - Parking','User - Name','Business - Neighborhoods','Business - Open?','Business - State','User - Compliments Cool','User - Compliments Cute','User - Compliments Funny','User - Compliments Hot','User - Compliments List','User - Compliments More','User - Compliments Note','User - Compliments Photos','User - Compliments Plain','User - Compliments Profile','User - Compliments Writer','User - Votes Cool','User - Votes Funny','User - Votes Useful']

Import Data

In [254]:
df = pd.read_csv('./Data/yelp_data.csv')
df = df[df.columns.difference(cols_to_remove)]

Preview Data

In [255]:
display(df.head())
print(df.shape)

,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since
0,True,beer_and_wine,casual,"Gluten-Free,Asian Fusion,Chinese,Restaurants",False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10
1,True,full_bar,casual,"Chinese,Restaurants",True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02
2,True,beer_and_wine,casual,"Italian,Pizza,Sandwiches,Restaurants",False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01
3,True,full_bar,dressy,"Steakhouses,Restaurants",False,Phoenix,False,True,False,True,cN6aBxe2mQvrQlzk26LyRQ,Durant's,average,3.0,418,4.0,False,True,True,True,no,33.477236,-112.073483,3/29/2005,KpOZEomlHF_tgJJXttQTwA,5,2,1,3,4.38,8,6VZNGc2h2Bn-uyuEXgOt5g,130,2005 2006 2007,2005-03
4,True,full_bar,casual,"Chinese,Restaurants",True,Phoenix,True,True,True,False,a8yzGF79KcfyRye2R6wt3Q,Golden Phoenix Chinese Restaurant,average,2.0,48,3.5,True,True,True,True,no,33.524850,-112.047889,4/7/2005,D9Dc32OAIpHUI3p7OFBWMQ,4,1,0,2,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01


(285764, 35)


Total restaurants

In [256]:
df['Business - Id'].unique().shape

(5696,)

Examine value counts and NAs and remove features with low variance or too many NAs.

In [6]:
print(df['Business - Ages Allowed'].isna().sum())
df['Business - Ages Allowed'].value_counts()

285285


21plus     431
allages     29
18plus      19
Name: Business - Ages Allowed, dtype: int64

In [7]:
print(df['Business - By Appointment Only'].isna().sum())
df['Business - By Appointment Only'].value_counts()

285217


False    538
True       9
Name: Business - By Appointment Only, dtype: int64

In [8]:
print(df['Business - Dietary Restrictions'].isna().sum())

279849


In [9]:
print(df['Business - Drive-Thru'].isna().sum())
df['Business - Drive-Thru'].value_counts()

260833


False    17798
True      7133
Name: Business - Drive-Thru, dtype: int64

In [10]:
print(df['Business - Open 24 Hours'].isna().sum())
df['Business - Open 24 Hours'].value_counts()

267126


False    18594
True        44
Name: Business - Open 24 Hours, dtype: int64

In [11]:
print(df['Business - Payment Types'].isna().sum())
df['Business - Payment Types'].value_counts()

283638


{u'amex': True, u'cash_only': False, u'mastercard': True, u'visa': True, u'discover': True}       1020
{}                                                                                                 793
{u'amex': False, u'cash_only': False, u'visa': True, u'mastercard': True, u'discover': False}      135
{u'amex': True, u'cash_only': False, u'visa': True, u'mastercard': True, u'discover': False}       101
{u'amex': False, u'cash_only': True, u'visa': False, u'mastercard': False, u'discover': False}      77
Name: Business - Payment Types, dtype: int64

In [12]:
cols_to_remove = ['Business - Payment Types', 'Business - Open 24 Hours', 'Business - Drive-Thru', 'Business - Dietary Restrictions', 'Business - Ages Allowed', 'Business - By Appointment Only', 'Business - Dogs Allowed', 'Business - Address', 'Business - Smoking', 'Business - Restaurant?', 'Business - Outdoor Seating', 'Business - Order at Counter', 'Business - Happy Hour', 'Business - Good For Kids', 'Business - Good For Dancing', 'Business - Corkage', 'Business - Coat Check', 'Business - BYOB/Corkage', 'Business - BYOB', 'Business - Monday Close','Business - Saturday Close','Business - Friday Open','Business - Saturday Open','Business - Monday Open','Business - Sunday Close','Business - Sunday Open','Business - Wednesday Close','Business - Thursday Close','Business - Wednesday Open','Business - Thursday Open','Business - Tuesday Close','Business - Tuesday Open','Business - Friday Close','Number of Records','Business - Accepts Insurance','Business - Parking','User - Name','Business - Neighborhoods','Business - Open?','Business - State','User - Compliments Cool','User - Compliments Cute','User - Compliments Funny','User - Compliments Hot','User - Compliments List','User - Compliments More','User - Compliments Note','User - Compliments Photos','User - Compliments Plain','User - Compliments Profile','User - Compliments Writer','User - Votes Cool','User - Votes Funny','User - Votes Useful']

In [13]:
df = df[df.columns.difference(cols_to_remove)]
display(df.head())
print(df.shape)

,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since
0,True,beer_and_wine,casual,"Gluten-Free,Asian Fusion,Chinese,Restaurants",False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10
1,True,full_bar,casual,"Chinese,Restaurants",True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02
2,True,beer_and_wine,casual,"Italian,Pizza,Sandwiches,Restaurants",False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01
3,True,full_bar,dressy,"Steakhouses,Restaurants",False,Phoenix,False,True,False,True,cN6aBxe2mQvrQlzk26LyRQ,Durant's,average,3.0,418,4.0,False,True,True,True,no,33.477236,-112.073483,3/29/2005,KpOZEomlHF_tgJJXttQTwA,5,2,1,3,4.38,8,6VZNGc2h2Bn-uyuEXgOt5g,130,2005 2006 2007,2005-03
4,True,full_bar,casual,"Chinese,Restaurants",True,Phoenix,True,True,True,False,a8yzGF79KcfyRye2R6wt3Q,Golden Phoenix Chinese Restaurant,average,2.0,48,3.5,True,True,True,True,no,33.524850,-112.047889,4/7/2005,D9Dc32OAIpHUI3p7OFBWMQ,4,1,0,2,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01


(285764, 35)


Recode and examine reviews by years.

In [14]:
df['ReviewYear'] = pd.DatetimeIndex(df['Review - Date']).year

In [15]:
display(df['ReviewYear'].value_counts())

2013    75071
2014    58946
2012    49378
2011    43506
2010    28689
2009    17055
2008     9566
2007     2896
2006      608
2005       49
Name: ReviewYear, dtype: int64

### Clean Business Categories

Explode cateogories

In [16]:
df['Business - Categories'] = df['Business - Categories'].str.split(",")

In [17]:
dfExp = df.explode('Business - Categories').copy()

Display top 20 categories for resutaurants

In [18]:
display(pd.DataFrame(dfExp['Business - Categories'].value_counts().iloc[1:]).head(20))

,Business - Categories
Nightlife,43428
American (New),43392
Bars,41424
Mexican,38665
American (Traditional),34386
Pizza,32477
Italian,30288
Breakfast & Brunch,27211
Sandwiches,27043
Food,24814


In [19]:
restVals = pd.DataFrame(dfExp['Business - Categories'].value_counts().iloc[1:].head(20))

Remove redundant categories

In [20]:
topList = restVals.index.tolist().copy()
topList.remove('Nightlife')
topList.remove('Food')
topList.remove('Pubs')
topList.remove('Sports Bars')
topList.remove('Bars')
print(topList)

['American (New)', 'Mexican', 'American (Traditional)', 'Pizza', 'Italian', 'Breakfast & Brunch', 'Sandwiches', 'Burgers', 'Sushi Bars', 'Chinese', 'Steakhouses', 'Japanese', 'Seafood', 'Mediterranean', 'Fast Food']


In [21]:
topCuisines = dfExp[dfExp['Business - Categories'].isin(topList)]

In [38]:
del dfExp

Restaurants Remaining

In [22]:
topCuisines['Business - Id'].unique().shape

(4818,)

In [23]:
print(topCuisines.shape)
topCuisines.head()

(336048, 36)


,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since,ReviewYear
0,True,beer_and_wine,casual,Chinese,False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10,2005
1,True,full_bar,casual,Chinese,True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02,2005
2,True,beer_and_wine,casual,Italian,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Pizza,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Sandwiches,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005


In [37]:
df = topCuisines.copy()

In [56]:
del topCuisines

### Clean Cities

In [39]:
df.loc[df['Business - City'] == 'Phoenix Sky Harbor Center', ['Business - City']] = 'Phoenix'
df.loc[df['Business - City'] == 'Pheonix', ['Business - City']] = 'Phoenix'

In [42]:
topCities = df['Business - City'].value_counts().head(10).index.tolist()

In [44]:
topCities = df.loc[df['Business - City'].isin(topCities)].copy()

In [45]:
print(topCities.shape)
topCities.head()

(320097, 36)


,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since,ReviewYear
0,True,beer_and_wine,casual,Chinese,False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10,2005
1,True,full_bar,casual,Chinese,True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02,2005
2,True,beer_and_wine,casual,Italian,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Pizza,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005
2,True,beer_and_wine,casual,Sandwiches,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005


In [57]:
df = topCities.copy()
del topCities

### Assign Zip Codes and correct cities

Get the zip code for each longitude and latitude

In [107]:
zipmap = df.drop_duplicates(subset=['Latitude','Longitude'])[['Business - Id', 'Latitude','Longitude']]

In [128]:
def get_zipcode(df, geolocator, lat_field, lon_field):
    try:
        location = geolocator.reverse((df[lat_field], df[lon_field]))
        #print(location.raw)
        return location.raw['address']['postcode']
    except:
        return None

In [132]:
zipmap['ZIP CODE'] = zipmap.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')

remove special delivery route numbers from zip codes (last 4 digits in 9 digit zip codes)

In [136]:
zipmap['ZIP CODE'] = zipmap['ZIP CODE'].str[:5]

add city to zip code

In [163]:
def getCity(df, zipCode):
    try:
        zipCode = df[zipCode]
        #print(zipCode)
        return zipcodes.matching(zipCode)[0]['city']
    except:
        return("NA")

In [ ]:
zipmap['cities']=zipmap.apply(getCity, axis=1, zipCode = 'ZIP CODE')

In [196]:
zipmap

,Business - Id,Latitude,Longitude,ZIP CODE,cities
0,ghzksdgNztTZA49PqMvPkA,33.330011,-111.790909,85269,Fountain Hills
1,hrOsdyai-9B91cGjBspHqw,33.335842,-111.859086,85225,Chandler
2,VVeogjZya58oiTxK7qUjAQ,33.449233,-112.065458,85004,Phoenix
3,cN6aBxe2mQvrQlzk26LyRQ,33.477236,-112.073483,85004,Phoenix
4,a8yzGF79KcfyRye2R6wt3Q,33.524850,-112.047889,85016,Phoenix
...,...,...,...,...,...
4010,q1QRFM3Rh8K9jPMWNus6hA,33.421531,-111.947576,85287,Tempe
4011,8K4pWvuJmGWXDgCugYKYvw,33.499192,-111.917816,85251,Scottsdale
4012,LsKF7GFQ0j5U1jLMwq0_hQ,33.474235,-112.074605,85003,Phoenix
4013,AstBY--d6rxLfrVIfXHF0g,33.480030,-112.080959,85013,Phoenix


In [202]:
zipmap.rename(columns={'Business - Id':'busID'}, inplace=True)

In [203]:
zipmap

,busID,Latitude,Longitude,ZIP CODE,cities
0,ghzksdgNztTZA49PqMvPkA,33.330011,-111.790909,85269,Fountain Hills
1,hrOsdyai-9B91cGjBspHqw,33.335842,-111.859086,85225,Chandler
2,VVeogjZya58oiTxK7qUjAQ,33.449233,-112.065458,85004,Phoenix
3,cN6aBxe2mQvrQlzk26LyRQ,33.477236,-112.073483,85004,Phoenix
4,a8yzGF79KcfyRye2R6wt3Q,33.524850,-112.047889,85016,Phoenix
...,...,...,...,...,...
4010,q1QRFM3Rh8K9jPMWNus6hA,33.421531,-111.947576,85287,Tempe
4011,8K4pWvuJmGWXDgCugYKYvw,33.499192,-111.917816,85251,Scottsdale
4012,LsKF7GFQ0j5U1jLMwq0_hQ,33.474235,-112.074605,85003,Phoenix
4013,AstBY--d6rxLfrVIfXHF0g,33.480030,-112.080959,85013,Phoenix


In [213]:
dfFinal = pd.merge(df, zipmap, how='left', on=['Latitude', 'Longitude'])

In [221]:
dfFinal = dfFinal[dfFinal.cities != "NA"]

In [228]:
TopcitiesList = dfFinal.cities.value_counts()[0:12].index.tolist()

In [230]:
dfFinal = dfFinal[dfFinal.cities.isin(TopcitiesList)]

In [235]:
dfFinal.head()

,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since,ReviewYear,busID,ZIP CODE,cities
0,True,beer_and_wine,casual,Chinese,False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10,2005,ghzksdgNztTZA49PqMvPkA,85269,Fountain Hills
1,True,full_bar,casual,Chinese,True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02,2005,hrOsdyai-9B91cGjBspHqw,85225,Chandler
2,True,beer_and_wine,casual,Italian,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005,VVeogjZya58oiTxK7qUjAQ,85004,Phoenix
3,True,beer_and_wine,casual,Pizza,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005,VVeogjZya58oiTxK7qUjAQ,85004,Phoenix
4,True,beer_and_wine,casual,Sandwiches,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005,VVeogjZya58oiTxK7qUjAQ,85004,Phoenix


In [237]:
dfFinal.to_csv("cleanDF.csv", index = False)

### Census hhi

In [238]:
hhi = pd.read_csv('AZ_Zip_Income_2014.csv')

In [240]:
hhi.rename(columns={'ZipCode':'ZIP CODE'}, inplace=True)
hhi.head()

,ZIP CODE,MeanIncome
0,85003,65184
1,85004,58495
2,85006,35167
3,85007,48918
4,85008,42852


In [247]:
hhi = hhi.astype({'ZIP CODE': 'str'})

In [251]:
dfFinal = pd.merge(dfFinal, hhi, how='left', on=['ZIP CODE'])

In [252]:
dfFinal.to_csv("cleanDF.csv", index = False)

In [253]:
dfFinal.head()

,Business - Accepts Credit Cards,Business - Alcohol,Business - Attire,Business - Categories,Business - Caters,Business - City,Business - Delivery,Business - Good For Groups,Business - Good for Kids,Business - Has TV,Business - Id,Business - Name,Business - Noise Level,Business - Price Range,Business - Review Count,Business - Stars,Business - Take-out,Business - Takes Reservations,Business - Waiter Service,Business - Wheelchair Accessible,Business - Wi-Fi,Latitude,Longitude,Review - Date,Review - Id,Review - Stars,Review - Votes Cool,Review - Votes Funny,Review - Votes Useful,User - Average Stars,User - Fans,User - Id,User - Review Count,User - Years Elite,User - Yelping Since,ReviewYear,busID,ZIP CODE,cities,MeanIncome
0,True,beer_and_wine,casual,Chinese,False,Gilbert,False,True,True,False,ghzksdgNztTZA49PqMvPkA,Pei Wei Asian Diner,loud,1.0,36,3.5,True,False,False,True,no,33.330011,-111.790909,2/1/2005,xSaWTOB24fK69MDNcjyuOg,4,0,0,0,4.00,0,gKhJyiCKkG6dHPNhr3dhDQ,1,NaN,2004-10,2005,ghzksdgNztTZA49PqMvPkA,85269,Fountain Hills,NaN
1,True,full_bar,casual,Chinese,True,Chandler,False,True,True,False,hrOsdyai-9B91cGjBspHqw,Tao Garden,average,2.0,55,4.0,True,False,True,True,no,33.335842,-111.859086,3/7/2005,I6Snobdz2kX_sqmzi-1K5A,3,0,0,0,3.80,4,kJc9YBRwmmZ_PG0uLHuEPQ,192,NaN,2005-02,2005,hrOsdyai-9B91cGjBspHqw,85225,Chandler,67543
2,True,beer_and_wine,casual,Italian,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005,VVeogjZya58oiTxK7qUjAQ,85004,Phoenix,58495
3,True,beer_and_wine,casual,Pizza,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005,VVeogjZya58oiTxK7qUjAQ,85004,Phoenix,58495
4,True,beer_and_wine,casual,Sandwiches,False,Phoenix,False,False,False,False,VVeogjZya58oiTxK7qUjAQ,Pizzeria Bianco,average,2.0,1289,4.0,False,False,True,True,no,33.449233,-112.065458,3/8/2005,KAYl3n-kUO7USLOMiywdDQ,5,3,0,0,3.55,2,K4FAia2Iy5MVnmBLfS-mCg,236,2006,2005-01,2005,VVeogjZya58oiTxK7qUjAQ,85004,Phoenix,58495


### Export Final Dataset

In [94]:
df.to_csv("cleanDF.csv")

### NLP

In [260]:
dfrev = pd.read_table('./Data/restaurant_bar_reviews_spring18.tsv', sep='\t')


In [261]:
dfrev.head()

,business_id,user_id,stars,average_stars,user_gender,gender_ratio,is_elite,user_reviews,yelping_since,review_date,useful,city,state,latitude,longitude,business_reviews,is_open,is_bar,is_restaurant,alcohol_level
0,L772e6l2Yd0DJEyCBxBNng,-1KKYzibGPyUX-MwkBTlrg,4,2.95,1,0.686983,True,163,2009-08-07,2014-09-19,1,Madison,WI,43.074776,-89.395948,167,1,1,1,6
1,pSQFynH1VxkfSmehRXlZWw,-3i9bhfvrM3F1wsC9XIB8g,5,3.89,1,0.997419,True,584,2005-08-07,2016-11-18,4,Phoenix,AZ,33.449160,-112.065635,1899,1,0,1,2
2,ghpFh6XpH1TYZhjAGdx-xw,-3i9bhfvrM3F1wsC9XIB8g,4,4.20,1,0.997419,True,584,2005-08-07,2016-11-18,3,Phoenix,AZ,33.483800,-112.075244,202,1,0,1,2
3,3qlqzQrwh8hjBltlgFgsJQ,-55DgUo52I3zW9RxkZ-EAQ,4,3.86,2,0.995774,False,31,2013-04-11,2015-11-11,0,Phoenix,AZ,33.459170,-112.068990,263,0,0,1,3
4,9O-L6F0cMfNmE5i07pN_xQ,-55DgUo52I3zW9RxkZ-EAQ,4,4.00,2,0.995774,False,31,2013-04-11,2016-08-13,0,Phoenix,AZ,33.626354,-112.066390,125,1,0,1,1


In [265]:
dfrev['Year'] = dfrev.review_date.str[:4]

In [269]:
dfrev = dfrev[dfrev.Year.astype('int64') <= 2014]

In [276]:
dfrev[dfrev.city.isin(dfFinal.cities.unique().tolist())]

,business_id,user_id,stars,average_stars,user_gender,gender_ratio,is_elite,user_reviews,yelping_since,review_date,useful,city,state,latitude,longitude,business_reviews,is_open,is_bar,is_restaurant,alcohol_level,Year
16,v9u0NgjA4iQeKGevFnYSCA,-GKEFg_92pp0q842clS_Jw,4,3.53,2,0.997851,True,305,2011-07-18,2013-12-02,2,Phoenix,AZ,33.523022,-112.023247,415,1,0,1,1,2013
17,uJe6qjWPRMgR7QIyZ_zTwQ,-GKEFg_92pp0q842clS_Jw,4,3.43,2,0.997851,True,305,2011-07-18,2013-12-02,0,Scottsdale,AZ,33.727396,-111.855560,133,0,1,1,6,2013
18,BSmqxPbQglQwP2R56OqOag,-GKEFg_92pp0q842clS_Jw,4,3.97,2,0.997851,True,305,2011-07-18,2014-02-27,0,Scottsdale,AZ,33.671310,-111.906005,378,1,0,1,3,2014
19,bCrMHwCLZzyKsAUOiAL0OA,-GKEFg_92pp0q842clS_Jw,4,3.38,2,0.997851,True,305,2011-07-18,2012-08-04,2,Scottsdale,AZ,33.626871,-111.893134,143,1,0,1,2,2012
20,ALgH4S65heUffUjcBiNJ0Q,-GKEFg_92pp0q842clS_Jw,4,3.06,2,0.997851,True,305,2011-07-18,2012-08-04,0,Scottsdale,AZ,33.672105,-111.890487,35,0,1,1,6,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072144,z83ABHR6E2N6eGLrm8OiEQ,zy5rvDRgPG1JUNbixZxwUQ,2,3.00,1,1.000000,True,788,2011-07-21,2013-08-14,0,Scottsdale,AZ,33.582505,-111.925112,8,0,0,1,1,2013
1072146,qKS7YV76te6olKnA_aMPTg,zz1vOjAa5nucyYBEMupuTg,4,4.12,2,0.998159,False,7,2010-07-06,2013-02-14,0,Scottsdale,AZ,33.502004,-111.928035,464,1,1,1,6,2013
1072147,wZukjLaf1V2dLRQap_Zriw,zzRq3Ykz8yfr2mcsPhYnEg,5,4.29,3,1.000000,False,13,2014-03-16,2014-03-16,0,Gilbert,AZ,33.355142,-111.789959,793,1,1,1,5,2014
1072148,IWOQNyRqWrnqQA5K49eK5g,zzRq3Ykz8yfr2mcsPhYnEg,4,3.48,3,1.000000,False,13,2014-03-16,2014-05-02,0,Tempe,AZ,33.423003,-111.940656,331,1,0,1,2,2014


### Report

After cleaning the data and doing most analysis in Tableau I came up with the following insights and conclusions.
- The highest rated restaurants don't necessarily equal the most popular ones, as review ammount it more indicative of how many customers a restaurant gets than review score
- There were siginificant differences between different restaurant categories and their popularity in different cities around the Phoenix area, although some foods like Mexican, American and Italian were usually popular all around
- The highest reviewed areas are closest to downtown phoenix and scottsdale, while lower reviewed places are in Surprise, Litchfield Park, and Wittman
- There are big discrepencies in review counts by price range, with slightly expensive restaurants (2) have the highest volume of reviews than all three other price range categories combined
- Users with more fans tend to have a lot more influence on the reviews they write, with their reviews obtaining more "votes" a metric that can be use as impressions
- Higher review scores (4 and 5) tend to have more review votes, meaning people take the time to read them thoroughly and react accordingly, and this trend seems to have lasted since 2005 to 2014
- The average household income of an area seems to affect the ratings restaurants get depending on their price range level